In [5]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File paths for the CSV files
school_data_to_load = Path("/Users/syedshahid/Downloads/Starter_Code 8/PyCitySchools/Resources/schools_complete.csv")
student_data_to_load = Path("/Users/syedshahid/Downloads/Starter_Code 8/PyCitySchools/Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# District Summary
# Total number of unique schools
total_schools = school_data_complete["school_name"].nunique()

# Total students
total_students = school_data_complete["Student ID"].nunique()

# Total budget
total_budget = school_data["budget"].sum()

# Average math score
average_math_score = school_data_complete["math_score"].mean()

# Average reading score
average_reading_score = school_data_complete["reading_score"].mean()

# Percentage passing math
passing_math_percentage = (school_data_complete["math_score"] >= 70).mean() * 100

# Percentage passing reading
passing_reading_percentage = (school_data_complete["reading_score"] >= 70).mean() * 100

# Overall passing percentage
overall_passing_percentage = (passing_math_percentage + passing_reading_percentage) / 2

# Creating District Summary DataFrame
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_percentage]
})

# School Summary
# Grouping students data by school name
students_grouped_by_school = school_data_complete.groupby("school_name")

# Total students per school
total_students_per_school = students_grouped_by_school.size()

# Average math score per school
average_math_score_per_school = students_grouped_by_school["math_score"].mean()

# Average reading score per school
average_reading_score_per_school = students_grouped_by_school["reading_score"].mean()

# Percentage passing math per school
passing_math_percentage_per_school = students_grouped_by_school["math_score"].apply(lambda x: (x >= 70).mean()) * 100

# Percentage passing reading per school
passing_reading_percentage_per_school = students_grouped_by_school["reading_score"].apply(lambda x: (x >= 70).mean()) * 100

# Overall passing percentage per school
overall_passing_percentage_per_school = (passing_math_percentage_per_school + passing_reading_percentage_per_school) / 2

# Adding school type and budget to the school summary
school_summary_df = school_data.set_index("school_name")[["type", "budget"]]

# Renaming columns for clarity
school_summary_df = school_summary_df.rename(columns={"type": "School Type", "budget": "Total School Budget"})

# Adding calculated metrics to the school summary
school_summary_df["Total Students"] = total_students_per_school
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"] / school_summary_df["Total Students"]
school_summary_df["Average Math Score"] = average_math_score_per_school
school_summary_df["Average Reading Score"] = average_reading_score_per_school
school_summary_df["% Passing Math"] = passing_math_percentage_per_school
school_summary_df["% Passing Reading"] = passing_reading_percentage_per_school
school_summary_df["% Overall Passing"] = overall_passing_percentage_per_school

# Highest-Performing Schools (by % Overall Passing)
top_schools_df = school_summary_df.sort_values("% Overall Passing", ascending=False).head(5)

# Lowest-Performing Schools (by % Overall Passing)
bottom_schools_df = school_summary_df.sort_values("% Overall Passing").head(5)

# Math Scores by Grade
math_scores_by_grade_df = school_data_complete.pivot_table(index="school_name", columns="grade", values="math_score", aggfunc="mean")

# Reading Scores by Grade
reading_scores_by_grade_df = school_data_complete.pivot_table(index="school_name", columns="grade", values="reading_score", aggfunc="mean")

# Scores by School Spending
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], bins=spending_bins, labels=spending_labels)
scores_by_spending_df = school_summary_df.groupby("Spending Ranges (Per Student)", observed=False).agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "% Passing Math": "mean",
    "% Passing Reading": "mean",
    "% Overall Passing": "mean"
})

# Scores by School Size
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins=size_bins, labels=size_labels)
scores_by_size_df = school_summary_df.groupby("School Size", observed=False).agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "% Passing Math": "mean",
    "% Passing Reading": "mean",
    "% Overall Passing": "mean"
})

# Scores by School Type
scores_by_type_df = school_summary_df.groupby("School Type").agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "% Passing Math": "mean",
    "% Passing Reading": "mean",
    "% Overall Passing": "mean"
})

# Displaying the DataFrames
print("District Summary:")
print(district_summary_df)
print("\nSchool Summary:")
print(school_summary_df)
print("\nTop Performing Schools (by % Overall Passing):")
print(top_schools_df)
print("\nLowest Performing Schools (by % Overall Passing):")
print(bottom_schools_df)
print("\nMath Scores by Grade:")
print(math_scores_by_grade_df)
print("\nReading Scores by Grade:")
print(reading_scores_by_grade_df)
print("\nScores by School Spending:")
print(scores_by_spending_df)
print("\nScores by School Size:")
print(scores_by_size_df)
print("\nScores by School Type:")
print(scores_by_type_df)


District Summary:
   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       74.980853          85.805463          80.393158  

School Summary:
                      School Type  Total School Budget  Total Students  \
school_name                                                              
Huang High School        District              1910635            2917   
Figueroa High School     District              1884411            2949   
Shelton High School       Charter              1056600            1761   
Hernandez High School    District              3022020            4635   
Griffin High School       Charter               917500            1468   
Wilson High School        Charter              1319574            2283   
Cabrera High School       Charter              1081356            1858